In [1]:
import pickle

In [2]:
PICKLE_PATH = "../app/board_games_pipeline.pkl"

In [3]:
# payload = {
#     "feature_df": df[FEATURE_COLS + ['name']],
#     "preprocessor": preprocessor,
#     "model": model,
#     # "pipeline": nn_pipeline,
# }

In [4]:
with open(PICKLE_PATH, 'rb') as pickle_file:
    payload = pickle.load(pickle_file)
feature_df = payload["feature_df"]
preprocessor = payload["preprocessor"]
model = payload["model"]
del payload

In [6]:
def get_game(df, game_name):
    name_match_mask_s = df['name'].str.lower() == game_name.strip().lower()
    matching_rows_df = df[name_match_mask_s]
    return matching_rows_df

In [18]:
def get_all_recommendations(preprocessor, model, df, game):
    game_features_df = game.drop(columns=['name'])
    game_features_preprocessed = preprocessor.transform(game_features_df) 
    distances, indices = model.kneighbors(game_features_preprocessed) 
    games = df.iloc[indices[0]]  # Select tracks corresponding to the nearest neighbors
    games["distance"] = distances[0]  # Add the distance of each neighbor as a new column

    # Step 10: Filter the columns for the final output
    cols = games.columns # you can explicitly choose to return specific columns here
    games = games.loc[:, cols]  # Keep the relevant columns
    games = games.sort_values(by="distance")  # Sort the tracks by their distance (most similar first)

    # Step 11: Return the recommended tracks as a list of dictionaries
    return games #.to_dict(orient="records")

In [22]:
game = get_game(feature_df, 'Catan')
all_recommended_games_df = get_all_recommendations(preprocessor, model, feature_df, game)

/var/folders/jf/2172hndd7zv4fznws878lj3h0000gn/T/ipykernel_76920/2172359522.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  games["distance"] = distances[0]  # Add the distance of each neighbor as a new column


In [35]:
def filter_recommendations(recommended_games_df, gamelist_length, min_players, max_players, min_playtime, max_playtime, min_age, 
                               min_average_rating):
    filtered_games_df = recommended_games_df[
        (recommended_games_df['max_players'] <= max_players) &
        (recommended_games_df['max_playtime'] <= max_playtime) &
        (recommended_games_df['min_age'] >= min_age) &
        (recommended_games_df['min_players'] >= min_players) &
        (recommended_games_df['min_playtime'] >= min_playtime) &
        (recommended_games_df['average_rating'] >= min_average_rating)
    ]
    filtered_limited_games_df = filtered_games_df.head(gamelist_length)
    return filtered_limited_games_df.to_dict(orient="records")

In [37]:
# import pandas as pd # for formatting not necessary in flask

# pd.DataFrame(
#     filter_recommendations(all_recommended_games_df, 7000, 2, 4, 60, 120, 4, 5) )

,max_players,max_playtime,min_age,min_players,min_playtime,playing_time,category,mechanic,average_rating,users_rated,...,mechanic_count,has_expansion,binned_playtime,binned_mechanics,binned_min_age,binned_category,len_description,description_sentiment,name,distance
0,4,120,10,3,60,120,negotiation,dice rolling,7.26569,67655,...,5,1,Long (61-120 minutes),Action & Movement Mechanics,Pre-Teen (10-12),Social/Party,457,0.098867,Catan,4.440892e-16
1,4,90,10,2,60,90,dice,dice rolling,7.63340,30432,...,3,1,Long (61-120 minutes),Card & Pool Mechanics,Pre-Teen (10-12),Children's/Family,198,0.128114,Stone Age,3.523874e-02
2,4,90,12,2,90,90,abstract strategy,area control / area influence,7.72379,20166,...,4,1,Long (61-120 minutes),Strategic Mechanics,Pre-Teen (10-12),Strategy,281,-0.060150,Tigris & Euphrates,1.098304e-01
3,4,60,10,2,60,60,humor,dice rolling,7.47467,20647,...,2,1,Medium (31-60 minutes),Randomness & Chance Mechanics,Pre-Teen (10-12),Entertainment/Pop Culture,530,0.072104,Galaxy Trucker,1.376223e-01
4,4,60,13,2,60,60,renaissance,card drafting,7.13008,15146,...,4,1,Medium (31-60 minutes),Action & Movement Mechanics,Teen (13+),Strategy,428,0.051123,Thurn and Taxis,1.529136e-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1117,4,60,6,2,60,60,abstract strategy,tile placement,5.86271,59,...,1,0,Medium (31-60 minutes),Pattern & Design Mechanics,Early Childhood (4-6),Strategy,60,0.080000,Nile,1.141094e+00
1118,4,60,8,2,60,60,electronic,roll / spin and move,5.50289,76,...,1,0,Medium (31-60 minutes),Action & Movement Mechanics,Middle Childhood (7-9),Unknown/Miscellaneous,114,-0.046270,Scene It? Harry Potter Second Edition,1.142268e+00
1119,4,60,8,2,60,60,economic,tile placement,6.07273,55,...,2,0,Medium (31-60 minutes),Game Structure Mechanics,Middle Childhood (7-9),Strategy,103,-0.114831,H2Olland,1.143190e+00
1120,2,60,12,2,60,60,modern warfare,hex-and-counter,6.00245,53,...,1,0,Medium (31-60 minutes),Unique Mechanics,Pre-Teen (10-12),Real-life/Simulation,45,-0.068750,Fast Attack Boats,1.146656e+00


In [40]:
def make_game_recommendations(preprocessor, model, feature_df, name, gamelist_length, min_players, max_players,
                              min_playtime, max_playtime, min_age, min_average_rating):
    game = get_game(feature_df, name)
    all_recommended_games_df = get_all_recommendations(preprocessor, model, feature_df, game)
    recommendations = filter_recommendations(all_recommended_games_df, gamelist_length, min_players, max_players,
                           min_playtime, max_playtime, min_age, min_average_rating)
    return recommendations

In [41]:
make_game_recommendations(preprocessor, model, feature_df, 'Catan', 7, 2, 4, 60, 120, 4, 5)

/var/folders/jf/2172hndd7zv4fznws878lj3h0000gn/T/ipykernel_76920/2172359522.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  games["distance"] = distances[0]  # Add the distance of each neighbor as a new column


[{'max_players': 4,
  'max_playtime': 120,
  'min_age': 10,
  'min_players': 3,
  'min_playtime': 60,
  'playing_time': 120,
  'category': 'negotiation',
  'mechanic': 'dice rolling',
  'average_rating': 7.26569,
  'users_rated': 67655,
  'category_count': 1,
  'mechanic_count': 5,
  'has_expansion': 1,
  'binned_playtime': 'Long (61-120 minutes)',
  'binned_mechanics': 'Action & Movement Mechanics',
  'binned_min_age': 'Pre-Teen (10-12)',
  'binned_category': 'Social/Party',
  'len_description': 457,
  'description_sentiment': 0.0988668194550547,
  'name': 'Catan',
  'distance': 4.440892098500626e-16},
 {'max_players': 4,
  'max_playtime': 90,
  'min_age': 10,
  'min_players': 2,
  'min_playtime': 60,
  'playing_time': 90,
  'category': 'dice',
  'mechanic': 'dice rolling',
  'average_rating': 7.6334,
  'users_rated': 30432,
  'category_count': 2,
  'mechanic_count': 3,
  'has_expansion': 1,
  'binned_playtime': 'Long (61-120 minutes)',
  'binned_mechanics': 'Card & Pool Mechanics',
 